In [1]:
from sys import displayhook
from binance.client import Client
from binance.enums import *
import pandas as pd
import time
from datetime import datetime
from sqlalchemy import create_engine
import numpy as np

def getminutedata(symbol, interval, start):
    frame = pd.DataFrame(client.get_historical_klines(symbol,
                                                        interval,
                                                        start + ' min ago UTC'))
    frame = frame.iloc[:,:6]                                                    
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit= 'ms')
    frame = frame.astype(float)
    return frame.reset_index()

if __name__ == "__main__":

    api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
    api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
    client = Client(api_key, api_secret)
    db_data = 'mysql+mysqldb://' + 'admin' + ':' + '0323A8E3DB' + '@' + 'coin-database-1-instance-1.c9hqcydvu19f.us-east-1.rds.amazonaws.com' + ':3306/' + 'crypto_db' + '?charset=utf8mb4'
    engine = create_engine(db_data)

    trend_table_status = pd.read_sql_query("show table status like 'crypto_trend'", engine)
    trend_table_rowcount_old = trend_table_status['Rows'].values[0]

    check_date = pd.read_sql_query("SELECT MAX(date_request) FROM crypto_trend", engine) 
    date_recent_checked = str(check_date.iloc[0, 0])


    cryptoString = 'SELECT * FROM crypto'
    crypto_table = pd.read_sql_query(cryptoString, engine)
    full_id_list = crypto_table.id.astype(int).tolist()
    #setup variables
    maxholds = 3
    trades = [[],[],[],[]]
    tradedata = []
    buyPrice  = [0] * (len(full_id_list))
    sellPrice = [0] * (len(full_id_list))
    tradeBuy  = [0] * (len(full_id_list))
    tradeSell = [0] * (len(full_id_list))
    buyTime   = [0] * (len(full_id_list))
    sellTime  = [0] * (len(full_id_list))
    holding = [False] * (len(full_id_list))
    buy_ids = []
    buy_threshold = 80
    sell_threshold = 50

    waiting = True
    check_data = False
    
    # need async here? or while loop with 2 functions
    while(True):

        if(waiting):
            if datetime.now().minute < 20:
                trend_table_status = pd.read_sql_query("show table status like 'crypto_trend'", engine)
                trend_table_rowcount_new = trend_table_status['Rows'].values[0]
                if (trend_table_rowcount_new > trend_table_rowcount_old):
                    check_data = True
                    waiting = False
                else:
                    time.sleep(5)
            else:
                time.sleep(60)

        if(check_data):
            date_last_checked = date_recent_checked
            new_trends_string = f"SELECT * FROM crypto_trend WHERE date_request > '{date_last_checked}' ORDER BY date_request, crypto_id, date"
            new_trends = pd.read_sql_query(new_trends_string, engine)
            #update to ensure that if the count increased inbetween the request for count and request for data, does not re-check
            trend_table_rowcount_old = pd.read_sql_query("show table status like 'crypto_trend'", engine)['Rows'].values[0]
            

            print(new_trends)
            if new_trends.empty:
                print(f"Dataframe empty for request: {new_trends_string}")
                check_data = False
                waiting = True
                continue

            date_recent_checked = new_trends.date_request.max()
            
            trends = new_trends.pivot(index="date", columns="crypto_id", values="hype") #unbeliiiiieveevevevable !! https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html    


            #check buying and selling conditions
            check_100 = ((trends.iloc[-1] == 100) | (trends.iloc[-2] == 100)) ##check trends with current time
            check_non0 = trends.iloc[0:-2].mean() > 7 
            check_peak = (trends.iloc[0:-10] < buy_threshold).all()
            trend_indicate_buy = np.logical_and.reduce((check_100, check_non0, check_peak), dtype=bool)

            check_data = False
            waiting = True


     crypto_id        date_request                date   hype
0           51 2022-03-06 16:02:03 2022-02-27 17:00:00    0.0
1           51 2022-03-06 16:02:03 2022-02-27 18:00:00    0.0
2           51 2022-03-06 16:02:03 2022-02-27 19:00:00    3.0
3           51 2022-03-06 16:02:03 2022-02-27 20:00:00    5.0
4           51 2022-03-06 16:02:03 2022-02-27 21:00:00    6.0
..         ...                 ...                 ...    ...
835        151 2022-03-06 16:02:04 2022-03-06 12:00:00   16.0
836        151 2022-03-06 16:02:04 2022-03-06 13:00:00   26.0
837        151 2022-03-06 16:02:04 2022-03-06 14:00:00   33.0
838        151 2022-03-06 16:02:04 2022-03-06 15:00:00   12.0
839        151 2022-03-06 16:02:04 2022-03-06 16:00:00  100.0

[840 rows x 4 columns]
     crypto_id        date_request                date  hype
0          102 2022-03-06 16:02:14 2022-02-27 17:00:00  28.0
1          102 2022-03-06 16:02:14 2022-02-27 18:00:00  38.0
2          102 2022-03-06 16:02:14 2022-02-27 19:

KeyboardInterrupt: 

Testing Buy settings:

In [1]:
from sys import displayhook
from binance.client import Client
from binance.enums import *
import pandas as pd
import time
from datetime import datetime
from sqlalchemy import create_engine
import numpy as np

def getminutedata(symbol, interval, start):
    frame = pd.DataFrame(client.get_historical_klines(symbol,
                                                        interval,
                                                        start + ' min ago UTC'))
    frame = frame.iloc[:,:6]                                                    
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit= 'ms')
    frame = frame.astype(float)
    return frame.reset_index()

api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
client = Client(api_key, api_secret)
db_data = 'mysql+mysqldb://' + 'admin' + ':' + '0323A8E3DB' + '@' + 'coin-database-1-instance-1.c9hqcydvu19f.us-east-1.rds.amazonaws.com' + ':3306/' + 'crypto_db' + '?charset=utf8mb4'
engine = create_engine(db_data)

trend_table_status = pd.read_sql_query("show table status like 'crypto_trend'", engine)
trend_table_rowcount_old = trend_table_status['Rows'].values[0]

check_date = pd.read_sql_query("SELECT MAX(date_request) FROM crypto_trend", engine) 
date_recent_checked = str(check_date.iloc[0, 0])


cryptoString = 'SELECT * FROM crypto'
crypto_table = pd.read_sql_query(cryptoString, engine)
full_id_list = crypto_table.id.astype(int).tolist()
#setup variables
maxholds = 3
trades = [[],[],[],[]]
tradedata = []
buyPrice  = [0] * (len(full_id_list))
sellPrice = [0] * (len(full_id_list))
tradeBuy  = [0] * (len(full_id_list))
tradeSell = [0] * (len(full_id_list))
buyTime   = [0] * (len(full_id_list))
sellTime  = [0] * (len(full_id_list))
holding = [False] * (len(full_id_list))
buy_ids = []
buy_threshold = 80
sell_threshold = 50

In [4]:
date_lower = "2022-03-09 08:02:25"
date_upper = "2022-03-09 08:10:09"


In [5]:
new_trends_string = f"SELECT * FROM crypto_trend WHERE date_request > '{date_lower}' AND date_request < '{date_upper}' ORDER BY date_request, crypto_id, date"
new_trends = pd.read_sql_query(new_trends_string, engine)
#update to ensure that if the count increased inbetween the request for count and request for data, does not re-check
trend_table_rowcount_old = pd.read_sql_query("show table status like 'crypto_trend'", engine)['Rows'].values[0]

if new_trends.empty:
    print(f"Dataframe empty for request: {new_trends_string}")
    

date_recent_checked = new_trends.date_request.max()

try:
    trends = new_trends.pivot(index="date", columns="crypto_id", values="hype") #unbeliiiiieveevevevable !! https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html
except:
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    print(new_trends)
    

#check buying and selling conditions
check_100 = ((trends.iloc[-1] == 100) | (trends.iloc[-2] == 100)) ##check trends with current time
check_non0 = trends.iloc[0:-2].mean() > 7 
check_peak = (trends.iloc[0:-10] < buy_threshold).all()
trend_indicate_buy = np.logical_and.reduce((check_100, check_non0, check_peak), dtype=bool)
curr_id_list = trends.columns.values
check_upward_ids = [curr_id_list[i] for i in [i for i, x in enumerate(trend_indicate_buy) if x]]


In [9]:
check_upward_ids

[16, 21, 63, 111, 172, 174, 181, 184, 254, 259]

In [10]:
print(f"Indicate buy ids: {check_upward_ids} between {date_lower} : {date_upper}")
for key, crypto_id in enumerate(check_upward_ids):
    value = getminutedata(crypto_table.loc[crypto_table['id'] == crypto_id]['ticker'].values[0], 
                                        '15m',
                                        '120')
    #add in splice to get data from selected time range
    close = value[['Time','Close']]                             
    close.index = pd.to_datetime(close['Time'])
    close = close.drop(columns=['Time'])
    close = close.rename(columns={"Close": f"{crypto_id}"})
    
    if (key != 0):
        closes = closes.join(close)
    else:
        closes = close

increase = closes.diff(periods=6)
check_increase = (increase.iloc[-1] >= 0)

indicate_buy = check_increase[check_increase].index.values
#print(f"Indicate buy ids: {indicate_buy} between {date_last_checked} : {date_recent_checked}")

curr_NOT_holding_ids = [full_id_list[i] for i in [i for i, x in enumerate(np.logical_not(holding)) if x]]
buy_ids = list(set(indicate_buy).intersection(curr_NOT_holding_ids))

Indicate buy ids: [16, 21, 63, 111, 172, 174, 181, 184, 254, 259] between 2022-03-09 08:02:25 : 2022-03-09 08:10:09


In [26]:
print(increase)

print(check_increase)

                         16    21      63    111   172    174  181    184  \
Time                                                                        
2022-03-09 11:00:00     NaN   NaN     NaN    NaN   NaN    NaN  NaN    NaN   
2022-03-09 11:15:00     NaN   NaN     NaN    NaN   NaN    NaN  NaN    NaN   
2022-03-09 11:30:00     NaN   NaN     NaN    NaN   NaN    NaN  NaN    NaN   
2022-03-09 11:45:00     NaN   NaN     NaN    NaN   NaN    NaN  NaN    NaN   
2022-03-09 12:00:00     NaN   NaN     NaN    NaN   NaN    NaN  NaN    NaN   
2022-03-09 12:15:00     NaN   NaN     NaN    NaN   NaN    NaN  NaN    NaN   
2022-03-09 12:30:00 -0.3771 -0.40 -0.0009 -0.419 -0.02 -0.003 -0.5 -0.104   
2022-03-09 12:45:00 -0.6242 -0.37 -0.0043 -0.345 -0.01 -0.004 -1.0 -0.066   

                        254     259  
Time                                 
2022-03-09 11:00:00     NaN     NaN  
2022-03-09 11:15:00     NaN     NaN  
2022-03-09 11:30:00     NaN     NaN  
2022-03-09 11:45:00     NaN     NaN  
2

In [54]:
check_increase['16'] = True
check_increase['174'] = True
#check_increase

list(set(check_increase[check_increase].index.values.astype(int)).intersection(curr_NOT_holding_ids))


AttributeError: 'numpy.ndarray' object has no attribute 'intersection'

In [43]:
set(check_increase[check_increase].index.values).intersection()

set()

In [52]:
set(check_increase[check_increase].index.values.astype(int))

{16, 174}

In [53]:
list(set(check_increase[check_increase].index.values.astype(int)).intersection(curr_NOT_holding_ids))

[16, 174]

In [38]:
curr_NOT_holding_ids

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [21]:
print([crypto_table.loc[crypto_table['id'] == crypto_id]['name'].values[0] for crypto_id in check_upward_ids])
print([crypto_table.loc[crypto_table['id'] == crypto_id]['ticker'].values[0] for crypto_id in check_upward_ids])

['Alpine F1 Team Fan Token', 'Anyswap', 'Chromia', 'Firo', 'Merit Circle', 'Mdex', 'Moonriver', 'NEAR Protocol', 'Syscoin', 'Alien Worlds']
['ALPINEUSDT', 'ANYUSDT', 'CHRUSDT', 'FIROUSDT', 'MCUSDT', 'MDXUSDT', 'MOVRUSDT', 'NEARUSDT', 'SYSUSDT', 'TLMUSDT']


In [58]:
crypto_table.loc[crypto_table['id'] == 286]['ticker']

285    WOOUSDT
Name: ticker, dtype: object

In [55]:
list(set([1,2,3,4,5,6,6,6,6,6]).intersection([6,1]))

[1, 6]

In [71]:
closes

,16,21,63,111,172,174,181,184,254,259
Time,,,,,,,,,,
2022-03-09 11:00:00,5.9852,12.88,0.3907,4.613,2.14,0.222,56.6,11.094,0.5305,0.1027
2022-03-09 11:15:00,6.2315,12.82,0.3920,4.550,2.14,0.223,56.8,11.064,0.5306,0.1025
2022-03-09 11:30:00,6.1528,12.81,0.3910,4.478,2.14,0.222,56.7,11.035,0.5310,0.1021
2022-03-09 11:45:00,5.7769,12.84,0.3907,4.624,2.16,0.221,56.6,11.031,0.5313,0.1024
2022-03-09 12:00:00,6.1200,12.74,0.3942,4.355,2.15,0.221,56.5,11.130,0.5330,0.1027
2022-03-09 12:15:00,5.7083,12.81,0.3951,4.200,2.15,0.222,56.9,11.142,0.5367,0.1031
2022-03-09 12:30:00,5.6081,12.48,0.3898,4.194,2.12,0.219,56.1,10.990,0.5273,0.1011
2022-03-09 12:45:00,5.6073,12.45,0.3877,4.205,2.13,0.219,55.8,10.998,0.5254,0.1013


In [62]:
closes.loc[closes.index[-1]]

16      5.6073
21     12.4500
63      0.3877
111     4.2050
172     2.1300
174     0.2190
181    55.8000
184    10.9980
254     0.5254
259     0.1013
Name: 2022-03-09 12:45:00, dtype: float64

In [70]:
import re
#print(re.match('/^\d{13}$/', "1234567891123"))
print(re.match('^(\d{13})$', "1234567891123"))


<re.Match object; span=(0, 13), match='1234567891123'>


In [73]:
check_decrease = np.logical_or((trends.iloc[-1] < sell_threshold), (trends.iloc[-2] < sell_threshold))
indicate_sell = check_decrease[check_decrease].index.values
#print(f"Indicate sell ids: {indicate_sell} between {date_last_checked} : {date_recent_checked}")

curr_holding_ids = [full_id_list[i] for i in [i for i, x in enumerate(holding) if x]]
sell_ids = list(set(indicate_sell).intersection(curr_holding_ids))



In [ ]:

# Buy control
if ((sum(holding) < maxholds) and buy_ids):
    for id in buy_ids:
        if (sum(holding) < maxholds):
            key = full_id_list.index(id)
            tradetime = closes.index[-1]
            buyPrice[key] = closes.loc[tradetime,str(id)]
            buyTime[key] = tradetime
            holding[key] = True
            print(f"Purchased {id} at {buyPrice[key]} - {tradetime}")
        else:
            continue
        


In [ ]:
# Sell control
if (sell_ids):
    for id in sell_ids:
        value = getminutedata(crypto_table.loc[crypto_table['id'] == id]['ticker'].values[0], 
                                            '15m',
                                            '120')
        close = value[['Time','Close']]                             
        close.index = pd.to_datetime(close['Time'])
        close = close.drop(columns=['Time'])
        # Sell execute function binance using ticker
        key = full_id_list.index(id)
        tradetime = close.index[-1]
        sellPrice[key] = close.loc[tradetime]
        sellTime[key] = tradetime
        print(f"Trade completed: id = {id} | open = {buyPrice[key]} at {buyTime[key]} | close = {sellPrice[key]} at {sellTime[key]} | profit = {(sellPrice[key]/buyPrice[key])}%")

        # upload to database
        quantity = 0
        trade = (id, buyTime[key], buyPrice[key], sellTime[key], sellPrice[key], str(sellTime[key] - buyTime[key]),  sellPrice[key]/buyPrice[key], quantity)
        index_names = ['crypto_id', 'date_buy', 'price_buy', 'date_sell', 'price_sell', 'trade_time', 'trade_return', 'quantity']
        result = pd.DataFrame([trade],columns=index_names)
        result.to_sql("crypto_trades", engine, if_exists="append", index=False)
        # tradedata.append([id, buyTime[key], sellTime[key], buyPrice[key], sellPrice[key], (sellPrice[key]/buyPrice[key]) ])
        holding[key] = False